# <center>Алгоритм Adaboost</center>
### <center>Кирилл Захаров</center>

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data_adaboost.csv', header=None)

**Задание.** Реализуйте алгоритм Adaboost.

В качестве базового алгоритма возьмем `DecisionTreeClassifier` глубины 1. Количество базовых алгоритмов $T=3$.

- В переменной `self.sample_weight` будем хранить текущие значения весов объектов.
- В методе `fit` последовательно будет производиться построение пней решения. На первом шаге алгоритма веса всех объектов одинаковы, т.е.:
$$w_i^0=\frac{1}{l}, i=1,\ldots,l,$$
где $l$ &ndash; количество объектов обучающей выборки.
- На каждом шаге алгоритма необходимо обучить пень решения с текущими весами объектов. Веса объектов задаются в методе `fit(X, y, sample_weight=...)` пня.
- После того, как пень обучен, вычислить:
$$\epsilon_t=\sum_{i=1}^{l} w_i[y_i \neq b_t(x_i)],$$
$$\alpha_t=\frac{1}{2}\log\frac{1-\epsilon_t}{\epsilon_t}.$$
- И обновить веса объектов:
$$w_i^t=w_i^{t-1}\exp(-\alpha_t y_i b_t(x_i)), i=1,\ldots,l,$$
$$w_i^t=\frac{w_i^t}{\sum_{j=1}^{l} w_j^t}.$$

- Обученные пни будем хранить в переменной `self.trees`.

1. Какими оказались веса объектов после обучения?
2. Пользуясь полученной композицией алгоритмов, классифицируйте точку (4,4) (метод `predict` в реализации).
3. За сколько шагов алгоритм классифицирует все точки верно, если глубина дерева равна 2?

In [55]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier

class AdaboostClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=3, max_depth=1, random_state=5):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []
        self.sample_weight = []
        self.coef = []
        
    def fit(self, X, y):
        self.sample_weight = np.array([1/len(X)] * 12)
        
        for _ in range(3):
            model = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            model.fit(X, y, sample_weight = self.sample_weight)
            
            prediction = model.predict(X)
            
            indx = np.where(y != prediction)[0]
            
            eps = sum(self.sample_weight[indx])
            alpha = 1/2 * np.log((1-eps)/eps)
            
            self.sample_weight = self.sample_weight * np.exp(alpha * y * prediction)
            self.sample_weight = self.sample_weight/sum(self.sample_weight)
            
            self.trees.append(model)
            self.coef.append(alpha)
        
    def predict(self, X):
        h = 0
        for t in range(3):
            h += self.trees[t].predict(X) * self.coef[t]
            
        return np.sign(h)    

In [56]:
adaboost = AdaboostClassifierCustom()

In [57]:
adaboost.fit(data.iloc[:,:2], data.iloc[:, 2])

In [62]:
adaboost.predict(pd.DataFrame([[4,4]]))

array([-1.])